## 1. Preprocessing the dataset:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


#Importing pre-requisite libaries
import os
import re
import nltk
import glob
import tqdm
import pickle
import random
import string
import inflect
import operator
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import tokenize
from nltk import word_tokenize
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from prettytable import PrettyTable
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/IR ASSIGNMENT 2/BBC News Train.csv',encoding='latin-1')

In [ ]:
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    # text = text.replace('x', '')
    text= text.replace(',','')
    text = text.replace('\'s','')
    text = text.replace('-',' ')
    return text
df['Text'] = df['Text'].apply(cleanText)

In [ ]:
tempdf=df['Text']
tempdf=tempdf.apply(word_tokenize)
df['Text']=tempdf
tempdf=df['Text']
nltk.download('stopwords')
stopli = set(stopwords.words('english'))
tempdf=tempdf.apply(lambda x: [item for item in x if item not in stopli])
df['Text']=tempdf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Use English stemmer.
stemmer = SnowballStemmer("english")
tempdf = tempdf.apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
df['Text']=tempdf

In [ ]:
df.head(5)

,ArticleId,Text,Category
0,1833,"[worldcom, ex, boss, launch, defenc, lawyer, d...",business
1,154,"[german, busi, confid, slide, german, busi, co...",business
2,1101,"[bbc, poll, indic, econom, gloom, citizen, maj...",business
3,1976,"[lifestyl, govern, mobil, choic, faster, bette...",tech
4,917,"[enron, boss, $, 168m, payout, eighteen, forme...",business


In [ ]:

df=df.drop('ArticleId', axis=1)

In [ ]:
df.head(10)

,Text,Category
0,"[worldcom, ex, boss, launch, defenc, lawyer, d...",business
1,"[german, busi, confid, slide, german, busi, co...",business
2,"[bbc, poll, indic, econom, gloom, citizen, maj...",business
3,"[lifestyl, govern, mobil, choic, faster, bette...",tech
4,"[enron, boss, $, 168m, payout, eighteen, forme...",business
5,"[howard, truant, play, snooker, conserv, leade...",politics
6,"[wale, silent, grand, slam, talk, rhys, willia...",sport
7,"[french, honour, director, parker, british, fi...",entertainment
8,"[car, giant, hit, merced, slump, slump, profit...",business
9,"[focker, fuel, festiv, film, chart, comedi, me...",entertainment


In [ ]:
count = 0
for word in df['Text'][0]:
  if word == 'worldcom':
    count+=1
print(count)

9


In [ ]:
df['Category'].unique()

array(['business', 'tech', 'politics', 'sport', 'entertainment'],
      dtype=object)

In [ ]:
def tf(dct, listOfWords):
    for word in listOfWords:
        if word not in dct:
          dct[word]=1
        else:
          dct[word]+=1
    return(dct)

In [ ]:
tfdf = pd.DataFrame({'Category':['business', 'tech', 'politics', 'sport', 'entertainment']})
tfdf.head()

,Category
0,business
1,tech
2,politics
3,sport
4,entertainment


In [ ]:
tfbusiness = {}
tftech = {}
tfpolitics = {}
tfsport = {}
tfentertainment = {}
for i in range(len(df['Text'])):
  if df['Category'][i]=='business':
    tfbusiness.update(tf(tfbusiness, df['Text'][i]))
  if df['Category'][i]=='tech':
    tftech.update(tf(tftech, df['Text'][i]))
  if df['Category'][i]=='politics':
    tfpolitics.update(tf(tfpolitics, df['Text'][i]))
  if df['Category'][i]=='sport':
    tfsport.update(tf(tfsport, df['Text'][i]))
  if df['Category'][i]=='entertainment':
    tfentertainment.update(tf(tfentertainment, df['Text'][i]))
tfdf = pd.DataFrame([tfbusiness, tftech, tfpolitics, tfsport, tfentertainment])
tfdf.head()

,worldcom,ex,boss,launch,defenc,lawyer,defend,former,chief,berni,...,103,playboyz,maniac,granada,cheeki,cassett,grandma,verbalici,stereophon,rapport
0,54.0,10,40,41,18,26,5,79,156,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2,5,121,9,8,9,18,46,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,35,5,40,16,19,35,82,58,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4,77,8,37,7,75,86,24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,10,2,25,1,15,3,52,16,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
tfdf.fillna(0, inplace = True)
tfdf.head()

,worldcom,ex,boss,launch,defenc,lawyer,defend,former,chief,berni,...,103,playboyz,maniac,granada,cheeki,cassett,grandma,verbalici,stereophon,rapport
0,54.0,10,40,41,18,26,5,79,156,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2,5,121,9,8,9,18,46,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,35,5,40,16,19,35,82,58,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,4,77,8,37,7,75,86,24,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,10,2,25,1,15,3,52,16,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
import math
icf = {}
for i in range(len(df['Text'])):
  for j in df['Text'][i]:
    if j in icf:
      if df['Category'][i] not in icf[j]:
        icf[j].append(df['Category'][i])
    else:
      icf[j] = []
      icf[j].append(df['Category'][i])
    # print(j)

for i in icf.keys():
  icf[i] = math.log(5/len(icf[i]), 10)
icf

{'worldcom': 0.6989700043360187,
 'ex': 0.0,
 'boss': 0.0,
 'launch': 0.0,
 'defenc': 0.0,
 'lawyer': 0.0,
 'defend': 0.0,
 'former': 0.0,
 'chief': 0.0,
 'berni': 0.6989700043360187,
 'ebber': 0.6989700043360187,
 'batteri': 0.0969100130080564,
 'fraud': 0.0969100130080564,
 'charg': 0.0,
 'call': 0.0,
 'compani': 0.0,
 'whistleblow': 0.6989700043360187,
 'first': 0.0,
 'wit': 0.0,
 '.': 0.0,
 'cynthia': 0.3979400086720376,
 'cooper': 0.0,
 'head': 0.0,
 'intern': 0.0,
 'account': 0.0,
 'alert': 0.0,
 'director': 0.0,
 'irregular': 0.22184874961635637,
 'practic': 0.0,
 'us': 0.0,
 'telecom': 0.3979400086720376,
 'giant': 0.0,
 '2002.': 0.0,
 'warn': 0.0,
 'led': 0.0,
 'collaps': 0.0,
 'firm': 0.0,
 'follow': 0.0,
 'discoveri': 0.0969100130080564,
 '$': 0.0,
 '11bn': 0.6989700043360187,
 '(': 0.0,
 'â£5.7bn': 0.6989700043360187,
 ')': 0.0,
 'mr': 0.0,
 'plead': 0.0,
 'guilti': 0.0,
 'conspiraci': 0.0969100130080564,
 'prosecut': 0.0,
 'argu': 0.0,
 'orchestr': 0.3979400086720376,
 'se

In [ ]:
tfdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Columns: 18305 entries, worldcom to rapport
dtypes: float64(16432), int64(1873)
memory usage: 715.2 KB


In [ ]:
for i in tfdf:
  tfdf[i] = tfdf[i]*icf[i]
tfdf.head()

,worldcom,ex,boss,launch,defenc,lawyer,defend,former,chief,berni,...,103,playboyz,maniac,granada,cheeki,cassett,grandma,verbalici,stereophon,rapport
0,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897,0.69897


## 2. The Dataset

In [ ]:
X = df.drop('Category', axis = 1)
print(X)

                                                   Text
0     [worldcom, ex, boss, launch, defenc, lawyer, d...
1     [german, busi, confid, slide, german, busi, co...
2     [bbc, poll, indic, econom, gloom, citizen, maj...
3     [lifestyl, govern, mobil, choic, faster, bette...
4     [enron, boss, $, 168m, payout, eighteen, forme...
...                                                 ...
1485  [doubl, evict, big, brother, model, capric, ho...
1486  [dj, doubl, act, revamp, chart, show, dj, duo,...
1487  [weak, dollar, hit, reuter, revenu, media, gro...
1488  [appl, ipod, famili, expand, market, appl, exp...
1489  [santi, worm, make, unwelcom, visit, thousand,...

[1490 rows x 1 columns]


## 3. Training the Naive Bayes classifier with TF-ICF:

In [ ]:
df_category = {'business':0, 'tech':1, 'politics':2, 'sport':3, 'entertainment':4}

In [ ]:
vectorized_df = pd.DataFrame()

In [ ]:
l1 =[]
lt = {}
for k in range(len(df['Text'])):
  lt = {}
  tokens = set(df['Text'][k])
  cat = df['Category'][k]
  if (cat == 'business'): cat = 0
  if (cat == 'tech'): cat = 1
  if (cat == 'politics'): cat = 2
  if (cat == 'sport'): cat = 3
  if (cat == 'entertainment'): cat = 4
  for j in tokens:
    lt[j] = tfdf[j][cat]
  l1.append(lt)

vectorized_df = pd.DataFrame(l1)
vectorized_df


,(,seri,agre,unawar,11bn,led,revenu,give,bankruptci,book,...,â£220m,6.13bn,â£2.89bn,â£234m,4gb,6gb,santi,unwelcom,phpbb,defac
0,0.0,0.0,0.0,0.38764,7.68867,0.0,0.0,0.0,27.855801,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1486,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1487,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,0.69897,0.69897,0.69897,0.69897,NaN,NaN,NaN,NaN,NaN,NaN
1488,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.69897,0.69897,NaN,NaN,NaN,NaN


In [ ]:
vectorized_df.fillna(0, inplace=True)
vectorized_df.tail()

,(,seri,agre,unawar,11bn,led,revenu,give,bankruptci,book,...,â£220m,6.13bn,â£2.89bn,â£234m,4gb,6gb,santi,unwelcom,phpbb,defac
1485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.69897,0.69897,0.69897,0.69897,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,0.69897,0.69897,0.00000,0.00000,0.00000,0.00000
1489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,3.49485,0.69897,2.09691,2.09691


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectorized_df.values, df['Category'].values, test_size=0.30, random_state=42)

### Implement the Naive Bayes classifier with the TF-ICF weighting scheme.

In [ ]:
from sklearn.naive_bayes import GaussianNB  
classifier = GaussianNB()  
classifier.fit(X_train, y_train)  

GaussianNB()

### Calculate the probability of each category based on the frequency of documents in the training set that belong to that category.

In [ ]:
count_bus = 0
count_tech = 0
count_politics = 0
count_sport = 0
count_ent = 0
for i in range(len(df['Text'])):
  if df['Category'][i] == 'business':
    count_bus += 1
  if df['Category'][i] == 'tech':
    count_tech += 1
  if df['Category'][i] == 'politics':
    count_politics += 1
  if df['Category'][i] == 'sport':
    count_sport += 1
  if df['Category'][i] == 'entertainment':
    count_ent += 1
total = len(df['Text'])
print('Probability of category Business: ', count_bus/total)
print('Probability of category Tech: ', count_tech/total)
print('Probability of category Politics: ', count_politics/total)
print('Probability of category Sport: ', count_sport/total)
print('Probability of category Entertainment: ', count_ent/total)

Probability of category Business:  0.22550335570469798
Probability of category Tech:  0.17516778523489934
Probability of category Politics:  0.18389261744966443
Probability of category Sport:  0.23221476510067113
Probability of category Entertainment:  0.18322147651006712


### Calculate the probability of each feature given each category based on the TF-ICF values of that feature in documents belonging to that category.

In [ ]:
prob_category = {0:count_bus/total, 1:count_tech/total, 2:count_politics/total, 3:count_sport/total, 4:count_ent/total}

In [ ]:
prob = tfdf.copy()
for i in range(len(prob)):
  prob.iloc[i] = prob.iloc[i]/prob_category[i]
prob.head()

,worldcom,ex,boss,launch,defenc,lawyer,defend,former,chief,berni,...,103,playboyz,maniac,granada,cheeki,cassett,grandma,verbalici,stereophon,rapport
0,167.378353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.796793,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,3.814891,3.814891,3.814891,3.814891,3.814891,3.814891,3.814891,3.814891,3.814891,3.814891


## 4. Testing the Naive Bayes classifier with TF-ICF:

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(y_pred, y_test))
print()
print(precision_score(y_pred, y_test, average=None))
print(recall_score(y_pred, y_test, average=None))
print(f1_score(y_pred, y_test, average=None))

0.9977628635346756

[0.99074074 1.         1.         1.         1.        ]
[1.         1.         1.         1.         0.98648649]
[0.99534884 1.         1.         1.         0.99319728]


## Improving the classifier

### 80:20

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectorized_df.values, df['Category'].values, test_size=0.20, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB  
classifier = GaussianNB()  
classifier.fit(X_train, y_train)  

GaussianNB()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(y_pred, y_test))
print(precision_score(y_pred, y_test, average=None))
print(recall_score(y_pred, y_test, average=None))
print(f1_score(y_pred, y_test, average=None))

1.0
[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1.]


### 50:50

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectorized_df.values, df['Category'].values, test_size=0.50, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB  
classifier = GaussianNB()  
classifier.fit(X_train, y_train)  

GaussianNB()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(y_pred, y_test))
print()
print(precision_score(y_pred, y_test, average=None))
print(recall_score(y_pred, y_test, average=None))
print(f1_score(y_pred, y_test, average=None))

0.9986577181208054

[0.99415205 1.         1.         1.         1.        ]
[1.         1.         1.         1.         0.99280576]
[0.99706745 1.         1.         1.         0.99638989]


### Try using different types of features such as n-grams or TF-IDF weights.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
ls = []
for i in range(len(df['Text'])):
  st = " ".join(df['Text'][i])
  ls.append(st)
X = vectorizer.fit_transform(ls)
X = X.toarray()
t_df = pd.DataFrame(X)
t_df

,0,1,2,3,4,5,6,7,8,9,...,17106,17107,17108,17109,17110,17111,17112,17113,17114,17115
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.024125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.018341,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0.0,0.033445,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1486,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1487,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1488,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 70:30

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(t_df.values, df['Category'].values, test_size=0.30, random_state=42)

In [ ]:
from sklearn.naive_bayes import GaussianNB  
classifier = GaussianNB()  
classifier.fit(X_train, y_train)  

GaussianNB()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(y_pred, y_test))
print()
print(precision_score(y_pred, y_test, average=None))
print(recall_score(y_pred, y_test, average=None))
print(f1_score(y_pred, y_test, average=None))

0.9172259507829977

[0.85185185 0.92405063 0.93023256 0.97029703 0.91780822]
[0.91089109 0.8902439  0.94117647 0.98       0.84810127]
[0.88038278 0.9068323  0.93567251 0.97512438 0.88157895]
